# Homework 2 - The Best Books of All Time

## Libraries

In [ ]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import re

from collections import defaultdict, Counter

filepath_books = "/content/drive/MyDrive/Datasets/Datasets-ADM-HW2/lighter_books.json"
filepath_authors = "/content/drive/MyDrive/Datasets/Datasets-ADM-HW2/lighter_authors.json"
filepath_list = "/content/drive/MyDrive/Datasets/Datasets-ADM-HW2/list.json"

# To display all columns and not only a sample
pd.set_option('display.max_columns', None)

In [ ]:
chunks = pd.read_json(filepath_books, lines = True, chunksize = 10000)

def reset_chunks(chunksize: int = 10000) -> None:
  global chunks
  chunks = pd.read_json(filepath_books, lines = True, chunksize = 10000)

In [ ]:
df_authors = pd.read_json(filepath_authors, lines = True)

---
# [RQ4]

1. **You should be sure there are no eponymous (different authors who have precisely the same name) in the author's dataset. Is it true?**

We have to check if authors with the same name have different ids.<br>
In our case there are 37 eponymous.

In [ ]:
# Get the columns 'id' and 'name' after counting the number of ids for each name
count = df_authors.groupby('name')['id'].nunique()
eponymous = count[count > 1].index
print('There are',eponymous.size,'eponymous')

## CODE to display the rows involving eponymous
## to use the function isin() we have to convert the series into a list
# epon_list = eponymous.astype(str).tolist()
# df_authors.loc[df_authors.name.isin(epon_list)]#

2. **Write a function that, given a list of author_id, outputs a dictionary where each author_id is a key, and the related value is a list with the names of all the books the author has written.**

We use a default dictionary to fill it while iterating and to add the dictionary keys on the fly.

In [ ]:
def books_from_authors(authors_ids: list) -> dict:
  """
  Given a list of author_id outputs a dictionary where each author_id is
  a key, and the related value is a list with the names of all the books the
  author has written

  Args:
      authors_ids (list[int]): list of autor_id

  Returns:
      book_ids (dict): dictionary with author_id and book's title
  """

  book_ids = defaultdict(list)

  # For every chunk of the dataset we look for the desired titles
  reset_chunks()
  for chk in chunks:
    # From each chunk we extract the records where author_id is in the list
    # given as input
    books = chk.loc[chk.author_id.isin(authors_ids)].reset_index()

    #Then we create the desired dictionary
    for index, row in books.iterrows():
      book_ids[row['author_id']].append(row['title'])

  return book_ids



Sample query for *J.K. Rowling* (id = 1077326) and *Douglas Adams* (id = 4)
```
research = books_from_authors([1077326, 4])
for key, item in research.items():
  print(key,': ',len(item),'books (counting all the editions)')
```
Gives output:
```
1077326 :  1914 books (counting all the editions)
4 :  1018 books (counting all the editions)
```



3. **What is the longest book title among the books of the top 20 authors regarding their average rating? Is it the longest book title overall?**

The longest book title from the top 20 authors is: <br>
"*God's Little Flock Healed: They Will Rise on Wings Like Eagles: They Will Run and Not Get Weary; They Will Walk and Not Grow Weak. Isaiah 40:31*"<br>
by Stellah Mupanduki

It's not the longest book title in the dataset. The maximum length for a title is 255. In the dataset there are a lot of books with such title length. We may suppose that this is a limitation of the database that truncates longer titles and adds triple dots at the end. The last script of this section shows that.

In [ ]:
# First we have to find the top 20 authors regarding their average rating
# Then we use the function books_from_authors() to retrieve their books and we
# find the one with the longest title
# TODO second part question

# First of all we count the number of authors with average_rating = 5 (the maximum)
authors_max_rating = df_authors.sort_values(by = 'average_rating', ascending = False).reset_index()[df_authors.average_rating == 5.0]
print("Authors with average rating = 5:", authors_max_rating.shape[0])

# They are too much, so we decided to consider only the top 20 authors for
# average_rating and for rating_counts
top_20_authors = df_authors.sort_values(by = ['average_rating', 'ratings_count'], ascending = False).reset_index().loc[:20]

books = books_from_authors(top_20_authors.id.astype(int).tolist())

# Then we look from the longest book title among them.
# If they are more then one then we save them all in a dictionary where
# the key is the author_id and as values the book titles
longest_book = defaultdict(list)
max_title = 0
for author_id, book_names in books.items():
  for name in book_names:
    if len(name) == max_title:
      longest_book[author_id].append(name)
    if len(name) > max_title:
      longest_book.clear()  # clear the dictionary
      max_title = len(name)
      longest_book[author_id].append(name)

print('\n\nLongest book title of the top 20 authors regarding their average rating:\n')
for id, names in longest_book.items():
  # It can happen that for one author we get a list with the same title multiple
  # times. This is due to the presence in the dataset of different editions of
  # the same book which have the same title
  # To avoid this we eliminate dupplicates using a set
  names_set = set(names)
  for name in names_set:
    print('\tAuthor :', top_20_authors[top_20_authors.id == id].reset_index().loc[0,'name'])
    print('\tTitle :',name,'\n')


Authors with average rating = 5: 17790


Longest book title of the top 20 authors regarding their average rating:

	Author : Stellah Mupanduki
	Title : God's Little Flock Healed: They Will Rise on Wings Like Eagles: They Will Run and Not Get Weary; They Will Walk and Not Grow Weak. Isaiah 40:31 



In [ ]:
# Find the longest book title LENGTH in the dataset

max_title = 0

reset_chunks()
for chk in chunks:
  max_chk = chk['title'].str.len().max()

  if max_chk > max_title:
    max_title = max_chk

print("Longest title length: ",max_title,"charachters")

Longest title length:  255 charachters


In [ ]:
# List the longest book titles in the first 10 chunks of the dataset
# We can observe that the maximal length is 255 and correspondin titles are
# truncated and finishes with triple dots

longest_title = defaultdict(list)
max_title = 0

i = 0
reset_chunks()
for chk in chunks:
  # From each chunk we extract the records where author_id is in the list
  # given as input
  books = chk.reset_index()

  for index, row in books.iterrows():
    title = row['title']
    if len(title) == max_title:
      longest_title[row['author_id']].append(title)
    if len(title) > max_title:
      longest_title.clear()  # clear the dictionary
      max_title = len(title)
      longest_title[row['author_id']].append(title)

  if i == 9:
    break
  else:
    i += 1

for id, titles in longest_title.items():
  titles_set = set(titles)
  for title in titles_set:
    print('Title :',title)
    print('Title length :',len(title),'\n')

Title : The New England Primer Issued Prior to 1830: A Bibliographical Checklist for the More Easy Attaining the True Knowledge of This Book, Embellished with a Hundred Cuts and Now Revised, Greatly Improved and Arranged in Two Alaphabets; With Preface, Introd...
Title length : 255 

Title : A Commentary and Review of Montesquieu's Spirit of Laws, Prepared for Press from the Original Manuscript in the Hands of the Publisher (1811): To Which Are Annexed, Observations on the Thirty-First Book, by the Late M. Condorcet. and Two Letters of Hel...
Title length : 255 

Title : Ensuring Compatibility with Enhanced 911 Emergency Calling Systems: A Progress Report: Hearing Before the Subcommittee on Telecommunications and the Internet of the Committee on Energy and Commerce, House of Representatives, One Hundred Seventh Congres...
Title length : 255 



4. **What is the shortest overall book title in the dataset? If you find something strange, provide a comment on what happened and an alternative answer.**

It does not exists a book with the shortest overall title in the dataset. Instead in the database there are many book whose title consists in only one charachter.<br>
In our research we also found out that there are books without a title. We counted them (33 books) and we decided to consider them as anomalies and to avoid them.<br>

The first script looks for the 5 shortest book titles in the first chunk of the dataset (books with multiple editions are considered only once). Since we found a lot of book titles consisting in only one alphanumeric charater, we decided to write a new script to count the books whose title length is 1 (avoiding the books where the author is "NOT A BOOK").<br><br>
The books whose title consists in only one charachter are 652.


In [ ]:
# Find the number of books whose title is an empty string

empty_titles = pd.DataFrame()

reset_chunks()
for chk in chunks:
  tmp = chk[chk['title'].str.strip() == '']
  if tmp.shape[0] > 0:
    empty_titles = pd.concat([empty_titles, tmp])

print("There are", empty_titles.shape[0],"books with an empty title")
print("Here a sapmle of books with empty title: ")
empty_titles.head()

There are 33 books with an empty title
Here a sapmle of books with empty title: 


,id,title,author_name,author_id,work_id,isbn,isbn13,asin,language,average_rating,rating_dist,ratings_count,text_reviews_count,publication_date,original_publication_date,format,edition_information,image_url,publisher,num_pages,series_id,series_name,series_position,shelves,description,authors
1995819,7857246,,Jacques André Naigeon,3418710,10995532,,2940009438689,,,0.00,5:0|4:0|3:0|2:0|1:0|total:0,0,0,,,,,https://s.gr-assets.com/assets/nophoto/book/11...,,,,,,[],,None
2046459,8065192,,Paul Scarron,670347,12755867,,2940010082123,,,0.00,5:0|4:0|3:0|2:0|1:0|total:0,0,0,,,,,https://s.gr-assets.com/assets/nophoto/book/11...,,,,,,[],,None
2052149,8086737,,Joseph Ennemoser,581668,12822154,,2940010001957,,,0.00,5:0|4:0|3:0|2:0|1:0|total:0,0,0,,,,,https://s.gr-assets.com/assets/nophoto/book/11...,,,,,,[],,None
2173750,8569155,,Johann Jakob Hess,3299662,13437816,,2940005095763,,,0.00,5:0|4:0|3:0|2:0|1:0|total:0,0,0,,,,,https://s.gr-assets.com/assets/nophoto/book/11...,,,,,,[],,None
2455300,9827732,,Aesop,12452,868263,,2940017505991,,,4.05,5:43293|4:40553|3:25752|2:4400|1:1082|total:11...,115080,2068,,-560,,,https://s.gr-assets.com/assets/nophoto/book/11...,,,,,,"[{'name': 'to-read', 'count': 49457}, {'name':...",,None


In [ ]:
 # Find the 5 SHORTEST book title LENGTH in the first chunk

shortest_titles = list()

reset_chunks(200000)
for chk in chunks:
  min_5_lengths = chk['title'].str.len().nsmallest(5).tolist()
  min_5_titles = chk[chk['title'].str.len().isin(min_5_lengths) & (chk['author_name']!='NOT A BOOK')]
  titles_list = min_5_titles[['title','author_name']].drop_duplicates(ignore_index=True)['title'].tolist()
  shortest_titles = shortest_titles + titles_list
  shortest_titles = sorted(shortest_titles, key = lambda x: len(x))
  shortest_titles = shortest_titles[0:min(5, len(shortest_titles))]
  break

print("The 5 shortest book titles:")
for title in shortest_titles:
  print('Title  :',title)

The 5 shortest book titles:
Title  : a
Title  : G
Title  : Q
Title  : π
Title  : 4


In [ ]:
# Find the book titles constisting in only one charachter

shortest_titles = pd.DataFrame()

reset_chunks()
for chk in chunks:
  min_titles = chk[(chk['title'].str.len() == 1) & (chk['author_name']!='NOT A BOOK')]
  shortest_titles = pd.concat([shortest_titles, min_titles])

In [ ]:
print('Number of books with titles consisting in only 1 charachter:',shortest_titles.shape[0])

Number of books with titles consisting in only 1 charachter: 652


---
# [RQ7]

1. **Estimate the probability that a book has over 30% of the ratings above 4.**

The result is 52.02%.

In our analysis we saw that the records that are not books do not contain the number of pages, or contain an empty string. We have also dropped the record with 'author_name' = 'NOT A BOOK'. The last that we did is to consider only the books whose 'rating_dist' is well formatted.

Since the rating distribution is discrete, i.e. can assume only integer values from 1 to 5 included, our probability coincides with the *probability that a book has over 30% of the ratings equals to 5*.

For every chunk of the data we choosed to use vectorization using the apply method. This allowed us to lower the execution time of the script.

In [ ]:
# First of all one should check that the total is correct. Should we do that?
# Alsi check that the total is the same as rating_counts?

# Check if is a book looking at num_pages (apart from format)

def parse_ratings(ratings: str) -> dict:
  """
  This funcion parses a string like '5:1674064|4:664833|3:231195|2:41699|1:16215|total:2628006'
  and returns a dictionary where the keys are '1','2','3','4','5','total' and the values are the
  corresponding rating counts
  """
  result = dict()
  # We check if the string is correctly formatted. Otherwise we return an
  # empty dictionary
  if re.match(r'^(\d+:\d+\|)*total:\d+$', ratings):
    ratings = ratings.split("|")
    for item in ratings:
      index, value = item.split(":")
      result[index] = int(value)

  return result

num_books_above_30 = 0
num_total_books = 0

reset_chunks()
for chk in chunks:
  tmp = chk[(chk.ratings_count > 0) & (chk.num_pages.notna()) & (chk.num_pages.str.strip() != '') & (chk.author_name != 'NOT A BOOK')][["id", "rating_dist"]]

  # Parse the string 'rating_dist' and transform it into a dictionary
  # We will add a new column to the temporary DataFrame
  tmp['rating_dict'] = tmp['rating_dist'].apply(parse_ratings)

  # Remove the records where the rating_dist field is empty or the total of the ratings is 0
  tmp = tmp[tmp['rating_dict'].apply(lambda x: (x != '') and 'total' in x and x['total'] > 0)]

  # Add the columns we will use later to compute the desired probability
  tmp['percentage_above_4'] = tmp.apply(lambda row: row['rating_dict']['5'] / row['rating_dict']['total'], axis = 1)
  tmp['rating_total'] = tmp.apply(lambda row: row['rating_dict']['total'], axis = 1)

  # Now we can use our data to update the variables we use to compute the probability.
  # Update the total books number (this is necessary because we have
  # filtered the rows that do not contain a book)
  num_total_books += tmp.shape[0]
  num_books_above_30 += tmp[tmp.percentage_above_4 >= 0.3].shape[0]

prob = num_books_above_30 / num_total_books * 100
print(f"The probability that a book has over 30% of ratings above 4 is: {prob: .2f}")

The probability that a book has over 30% of ratings above 4 is:  52.02


2. **Estimate the probability that an author publishes a new book within two years from its last work.**

To compute this probability we want to evaluate the ratio

$$\frac{\sum_{\text{author}}\text{\# of intervals} \neq 0 \text{ of books published within two years by that author }}{\sum_{\text{author}}\text{\# of intervals} \neq 0 \text{ among consecutive publications by that author}}$$
<br>

We assume that a book must have a number of pages that is neither NaN or an empty string. In this way we try to not consider records with 'format' like 'Audio CDs' or 'MP3'.

For the same reason we consider only those books whose 'original_publication_date' is not empty and does not raise an error while trying to convert it to a DateTime format. We found that some date are negative and we decided to not consider those records.

Our strategy begins with, once retrieved the useful attributes, ordering the DataFrame by 'author_id' and 'original_publication_date'. Then we compute all the date differences by consecutive publications of the same author.
We choosed remove the records with time difference = 0 because:
* we are interested in intervals
* we do not consider the authors that have published only one book

At this point we can count the books that have pubblication date difference within two years (730 days) and the total number of date differences among consecutive publications.


The probability that an author publishes a new book within two years from its last work is 20.87%.

In the end we choose to write a function that, given an author_id, calculates the probability that he/her publishes a new book within two years. In return 0 also for authors that have published only one book.

In [ ]:
# First of all we filter the books dataset and we consider only the
# columns that we need

books = pd.DataFrame()

reset_chunks()
for chk in chunks:
  # Consider only the records with valid num_pages and original_pubblication_date that is not empty
  tmp = chk[(chk.num_pages.notna()) & (chk.num_pages.str.strip() != '') & (chk.original_publication_date.str.strip() != '')][['id','author_id','original_publication_date']]
  books = pd.concat([books, tmp], ignore_index = True)

# We find the ids of books with original_publication_date that is not convertible to DateTime
incorrect_dates = books[pd.to_datetime(books['original_publication_date'], errors='coerce').isna()]
incorrect_dates_id = incorrect_dates['id'].tolist()

# Drop incorrect records from our dataframe
books = books[~books['id'].isin(incorrect_dates_id)]


In [ ]:
# Now we can convert the date to a correct DateTime format
# Records with only the year are set by default month=January and day=01
books['original_publication_date'] = pd.to_datetime(books['original_publication_date'])

# We order the dataframe by author_id and original_publication_date
books = books.sort_values(by = ['author_id','original_publication_date'], ascending = True)

# Here we calculate all the differences between two consecutive publications
# of each single author, using a function and apply()
def calculate_intervals(grouped_by_author):
    return grouped_by_author['original_publication_date'].diff()

# Apply the custom function to each author group
books['pub_date_interval'] = books.groupby('author_id').apply(calculate_intervals).reset_index(level = 0, drop = True)

total = books.shape[0]

# Now we remove the books where pub_date_interval = 0. This is done for a correct
# computation of the probability. We remove books published in the same day.
# In this way also remove the authors that have published only one book
# (and so they cannot have published two books within two years).
books = books[books['pub_date_interval'] > pd.Timedelta(days=0)]

# Now we get, grouped by author, the number of books that have been published within two years
# We remove the entries where the interval is zero, which corresponds to publications in the same day,
# that we assume to be impossible

books_within_2years = books[books['pub_date_interval'] <= pd.Timedelta(days=730)]

# Evaluate the desired probability
prob = books_within_2years.shape[0] / total  * 100

print("The number of books published within two years: ", books_within_2years.shape[0])
print("The probability that an author publishes a new book within two years from its last work: ", round(prob, 2),"%")

The number of books published withintwo years:  788498
The probability that an author publishes a new book within two years from its last work:  20.87 %


In [ ]:
def prob_within_2years(author_id: int) -> float:
  """
  Given an author_id, calculates the probability that he/her publishes a
  new book within two years. In return 0 also for authors that have published
  only one book.
  """
  books = pd.DataFrame()

  reset_chunks()
  for chk in chunks:
    # Consider only the records with valid num_pages and original_pubblication_date that is not empty
    tmp = chk[(chk.num_pages.notna()) & (chk.num_pages.str.strip() != '') & (chk.original_publication_date.str.strip() != '') & (chk.author_id == author_id)][['id','author_id','original_publication_date']]
    books = pd.concat([books, tmp], ignore_index = True)

  # We find the ids of books with original_publication_date that is not convertible to DateTime
  incorrect_dates = books[pd.to_datetime(books['original_publication_date'], errors='coerce').isna()]
  incorrect_dates_id = incorrect_dates['id'].tolist()

  # Drop incorrect records from our dataframe
  books = books[~books['id'].isin(incorrect_dates_id)]

  if books.shape[0] == 0: return 0

  # Now we can convert the date to a correct DateTime format
  # Records with only the year are set by default month=January and day=01
  books['original_publication_date'] = pd.to_datetime(books['original_publication_date'])

  # We order the dataframe by author_id and original_publication_date
  books = books.sort_values(by = ['author_id','original_publication_date'], ascending = True)

  # Here we calculate all the differences between two consecutive publications
  books['pub_date_interval'] = books['original_publication_date'].diff().reset_index(level = 0, drop = True)

  total = books.shape[0]

  books = books[books['pub_date_interval'] > pd.Timedelta(days=0)]

  if books.shape[0] == 0: return 0

  books_within_2years = books[books['pub_date_interval'] <= pd.Timedelta(days=730)]

  # Evaluate the desired probability
  prob = books_within_2years.shape[0] / total  * 100

  return prob

In [ ]:
# For example we test the function prob_within_2years() with
# 'Douglas Adams' that has author_id = 4 (he's dead, but in the database there
# is no information about it)
douglas_prob = prob_within_2years(4)
print('The probability that Douglas Adams publishes a new book within two years from its last work is: ', round(douglas_prob, 2))

The probability that Douglas Adams publishes a new book within two years from its last work is:  3.61%


3. **In the file list.json, you will find a peculiar list named "The Worst Books of All Time." Estimate the probability of a book being included in this list, knowing it has more than 700 pages.**

To evaluate this probability we have to use the conditional probability. In other word:

$$\mathbb{P}(\text{a book is in 'The Worst Books of All Time'} | \text{the book has more than 700 pages}) =\\ =  \frac{\text{joint probability}}{\text{marginal probability}}$$
<br> where
$$\text{joint probability} = \mathbb{P}(\text{a book is in 'The Worst Books of All Time'}\cap \text{the book has more than 700 pages})$$
<br>and
$$\text{marginal probability} = \mathbb{P}(\text{a book has more than 700 pages})$$

The derired probability is 0.002%. <br>This means that among the books with more than 700 pages, only 1 out of 50000 is in 'The Worst Books of All TIme' list.

In [ ]:
# First of all we retrive the books inside 'The Worst Books of All Time'
# It is the first record (the one with id == 2) inside the 'list.json' dataset

worst_book_record = pd.Series(dtype = object)

list_chunks = pd.read_json(filepath_list, lines = True, chunksize = 10)
for chk in list_chunks:
  worst_books_record = chk.loc[0]
  break

# List containing the worst books' ids
books_json = worst_books_record['books']
print("Number of books in 'The Worst Books of All Time' list:", len(books_json))

# Creating a list with all the book ids as integer
worst_book_ids = [int(item['book_id']) for item in books_json]

Number of books in 'The Worst Books of All Time' list: 7393


In [ ]:
# Now we look into the books daatset and we save a DataFrame with 'id' and
# 'num_pages' of the worst books
worst_books = pd.DataFrame(columns = ['id','num_pages'])

pages_list = list()

reset_chunks()
for chk in chunks:
  tmp = chk[chk['id'].astype(int).isin(worst_book_ids)][['id','num_pages']]
  worst_books = pd.concat([worst_books, tmp])

  # We also save the number of pages for all books, to evaluate the probability.
  # We do not consider the books with empty number of pages
  pages_list += chk[chk['num_pages'].str.strip() != '']['num_pages'].tolist()


# The probability that a book is in the Worst Books List AND has more than 700 pages.
# First of all we remove records with invalid num_pages
worst_books = worst_books[worst_books['num_pages'].astype(str).str.strip() != '']
num_worst_moreThan700 = worst_books[worst_books['num_pages'].astype(int) > 700].shape[0]
prob_intersection = num_worst_moreThan700 / len(pages_list) * 100

print(f'The joint probability is: {round(prob_intersection,3)}%')

# The probability that a books has more than 700 pages
num_moreThan700 = len([x for x in pages_list if x > 700])
prob_moreThan700 = num_moreThan700 / len(pages_list) * 100

print(f'The marginal probability is: {round(prob_moreThan700,2)}%')

print(f'\nThe probability of a book being included in this list, knowing it has more than 700 pages, is: {round(prob_intersection / prob_moreThan700, 4 )}%')

The joint probability is: 0.006%
The marginal probability is: 2.81%

The probability of a book being included in this list, knowing it has more than 700 pages, is: 0.002%


4. **Are the events X=’Being Included in The Worst Books of All Time list’ and Y=’Having more than 700 pages’ independent? Explain how you have obtained your answer.**

From the probability theory we know that the two events are independent if and only if

$$\mathbb{P}(X \cap Y) = \mathbb{P}(X)\mathbb{P}(Y)$$

We have that:
*   $\mathbb{P}(X) \approx 0.187 $
*   $\mathbb{P}(Y) \approx 2.81 $
*   $\mathbb{P}(X \cap Y) \approx  0.00556$
*   $\mathbb{P}(X)\mathbb{P}(Y) \approx  0.525$

Since $\mathbb{P}(X \cap Y) \neq \mathbb{P}(X)\mathbb{P}(Y)$ we found that $X$ and $Y$ are not independent.



In [ ]:
prob_x = len(worst_book_ids) / len(pages_list) * 100
print('P(X) = ',round(prob_x, 8))
print('P(Y) = ',round(prob_moreThan700, 8))
print('\nP(X)P(Y) = ',round(prob_x*prob_moreThan700, 8))
print('P(X intersection Y) = ',round(prob_intersection, 8))

P(X) =  0.18689977
P(Y) =  2.80918476

P(X)P(Y) =  0.525036
P(X intersection Y) =  0.00556174


---
# [CLQ]

Our script __`commandline_original.sh`__ contains:

```
#!/bin/bash
jq -c '. | {id: (.id), title: (.title), total_books_count: (.works | map(.books_count | tonumber) | add)}' series.json >> sum.json
jq -c -s 'sort_by(.total_books_count) | reverse[]' sum.json >> ordered.json
sed -n '1,5p' ordered.json | jq .
rm sum.json ordered.json

```
The first line is not part of the script, but instruct the operating system to use the specified interpreter. In this case it's the common shell.<br>
We use the bash command jq to parse the `series.json` file and to filter it.


The second line does the following operations:
1. from the `series.json` file outputs a json formatted text where each object has the following keys: '**id**', '**title**', '**total_books_count**'. The first two keys are the same of the original serie object, while the third is computed in the following way. For each works does the following:
> * first of all we use the map function to convert the '**books_count**' values to numbers (because the `series.json` file provides strings)  
> * then we add all the '**book_counts**' of the same work
> * we save the result of the addition into the field '**total_books_count**'
2. save the text formatted as json into a temporary file called `sum.json`

The third line parses the file `sum.json` and does the following operations:
> * sort the json objects with regard to the field '**total_books_count**'
> * reverse the sorted object (because the default sorting is in ascending order)
> * output the result in a temporary file named `ordered.json`

The fourth line prints to the screen (formated as json) the first 5 objects of the file `ordered.json` (which are the top 5 series with the highest total '**books_count**' among all of their associated books). More specifically we are printing the first 5 lines of that file, where each line is a single object.

The last lines delete the temporary files `sum.json` and `ordered.json`.


\\
Then we tried to get a more robust script implementation from ChatGPT. Our __commandline_LLM.sh__ script contains the following two lines:

```
#!/bin/bash
cat series.json | jq -c '. | {id: .id, title: .title, total_books_count: (.works | map(.books_count | tonumber) | add)}' | jq -s 'sort_by(-.total_books_count) | .[:5]'
```
We obtained it with the following three queries:
1. *The file series.json contains a list of book series. In each series's 'works' field there is a list of books that are part of that series, each book has a field 'books_count' . For each work sum the 'books_count'. Report the the top 5 series with the highest total 'books_count' among all of their associated books. The output should be a list of the five json objects with fields 'id','title','total_books_count'. Use command line tools*<br>
As result we received
```
cat series.json | jq -c '.[] | {id: .id, title: .title, total_books_count: (.works | map(.books_count) | add)}' | jq -s 'sort_by(-.total_books_count) | .[:5]'
```
We manually changed `jq -c '.[]` with `jq -c '.` in order to let jq reading correctly all objects.
2. since we got an error we queried again the LLM with the error: *jq: error (at <stdin>:226659): string ("117826822") cannot be negated* and the LLM printed the final script.


Both the result and the LLM script are correct. We claim it's correctness by comparing it with our original script. They do the same thing but with different filters. The ChatGPT script is, without any doubt, faster then our because avoids to write temporary files to the disk. This operation would be very slow. The addition part is the same as in our script. Then it sorts the 'total_books_count' in descending order by simply adding a minus sign as prefix. In the end it prints to the screen the result in a more efficient way, which is by simply considering the first five sorted objects.


We compared the execution time of the two scripts on the same local Ubuntu machine:
> * __`commandline_original.sh`__ : ~20 seconds
> * __`commandline_LLM.sh`__ : ~17 seconds

Below are the output of the two scripts:

![Our script](images/CLQ_original.png)
![LLM script](images/CLQ_LLM.png)

---
# [Bonus 1]

1. *Select one alternative library to Pandas (i.e., Dask, Polar, Vaex, Datatable, etc.), upload authors.json dataset, and filter authors with at least 100 reviews. Do the same using Pandas and compare performance in terms of milliseconds.*

We chose to used Dask.

From the following script we had the following results:
* the execution time for Pandas is: `20254 ms`
* the execution time for Dask is:   `40743 ms`

This is interesting because we know that Dask is faster. But in this case it is slower because the dataset is smaller than the memory size. Pandas is optimized for that and takes less time to execute it.

On the other hand, if we have to process datasets whose dimension is bigger than the available memory then Dask is faster. Moreover, Dask can be executed on a cluster of server to increase the speed of the task.

In [ ]:
import time
import dask.dataframe as dd

start_time = time.time()

# Load the dataset with Dask
ddf_authors = dd.read_json(filepath_authors, lines = True)

ddf_filtered = ddf_authors[ddf_authors.text_reviews_count >= 100]

authors_more_100_reviews = ddf_filtered.shape[0].compute()

exec_time = time.time() - start_time
print('Authors with at least 100 reviews: ', authors_more_100_reviews)
print('Dask performance (in milliseconds): ', round(exec_time * 1000))

Authors with at least 100 reviews:  64565
Dask performance (in milliseconds):  40743


In [ ]:
import time

start_time = time.time()

# Load the dataset with Pandas
df_authors = pd.read_json(filepath_authors, lines = True)

df_filtered = pd.DataFrame()

df_filtered = df_authors[df_authors.text_reviews_count >= 100]

authors_more_100_reviews = df_filtered.shape[0]

exec_time = time.time() - start_time
print('Authors with at least 100 reviews: ', authors_more_100_reviews)
print('Pandas performance (in milliseconds): ', round(exec_time * 1000))

Authors with at least 100 reviews:  64565
Pandas performance (in milliseconds):  20254


2. **Select one alternative library to Pandas (i.e., Dask, Polar, Vaex, Datatable, etc.), upload books.json, and join them with authors.json based on author_id. How many books don’t have a match for the author?**

For this question we used the library *PySpark*.

We tried to use Dask but we where not able to find out a solution for the exception: "*ValueError: The columns in the computed data do not match the columns in the provided metadata
Order of columns does not match dask*"

We found out that there no books without a match for the author.

In [ ]:
!pip install -q pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("BonusQuestion").getOrCreate()

# Load the two datasets
sparkdf_books = spark.read.json(filepath_books)
sparkdf_authors = spark.read.json(filepath_authors)

# We rename the 'id' column to 'author_id' in order to be able
# to join the two datasets
sparkdf_authors = sparkdf_authors.withColumnRenamed('id', 'author_id')

sparkdf_joined = sparkdf_books.join(sparkdf_authors, "author_id", "inner")

# Now we can count the number of rows we are interested in
num_books = sparkdf_books.count()
num_books_match = sparkdf_joined.count()

# Calculate the number of unmatched books
num_books_no_match = num_books - num_books_match

print("The number of books without a match for the author is: ", num_books_no_match)

# Release the resources occupied the pyspark session
spark.stop()

The number of books without a match for the author is:  0


---
# [Bonus 2]

We chose to mine the author dataset.

First of all we queried ChatGPT for a list of literary genres and, for each genre, a list of the most common words associated to that genre.

For the authors we looked for the literary genre inside the 'about' field. We created a DataFrame where the indexes are the author ids and the columns the literary genres. That we filled this DataFrame simply counting how many times each genre name appears inside the 'about' field.

Then we wrote a function that, given an author id, gives as output the genre name (or a list if more than one) of the most common genre (or genres) used to describe the author.

In the end we add a column to the original author DataFrame called 'literary_genre', that contains a list with the associated genre names.  

In [ ]:
from collections import Counter

# Dictionary obtained using ChatGPT of the most common words used
# for the most common literary genres
literature_genres_dict = {
    "Fiction": ["Protagonist", "Plot", "Character", "Setting", "Conflict", "Theme"],
    "Mystery": ["Detective", "Clue", "Suspense", "Alibi", "Whodunit", "Red Herring"],
    "Science Fiction": ["Futuristic", "Technology", "Aliens", "Spaceship", "Dystopian", "Speculative"],
    "Fantasy": ["Magic", "Mythical", "Dragons", "Wizards", "Quest", "Enchantment"],
    "Romance": ["Love", "Relationship", "Passion", "Heartbreak", "Chemistry", "Affection"],
    "Historical Fiction": ["Era", "Historical", "Authenticity", "Period", "Setting", "Anachronism"],
    "Adventure": ["Quest", "Exploration", "Adrenaline", "Hero", "Challenge", "Journey"],
    "Horror": ["Fear", "Terror", "Suspense", "Supernatural", "Haunting", "Nightmare"],
    "Drama": ["Conflict", "Tension", "Emotion", "Tragedy", "Dialogue", "Intensity"],
    "Poetry": ["Metaphor", "Rhyme", "Stanza", "Imagery", "Verses", "Rhythm"],
    "Comedy": ["Humor", "Laughter", "Satire", "Wit", "Parody", "Irony"],
    "Biography": ["Life", "Autobiography", "Memoir", "Achievements", "Personal", "Influence"]
}

# Creating a list only for literature genres
literature_genres = [x.lower() for x in literature_genres_dict.keys()] + ['novel']

all_words = [word.lower() for key, lst in literature_genres_dict.items() for word in lst]
all_words.sort()
all_words = list(set(all_words))

In [ ]:
# Loading the dataset and saving the authors 'id' and 'about' columns in an other
# DataFrame for a faster computation
df_authors = pd.read_json(filepath_authors, lines = True)
df_authors_about = df_authors[['id','about']]

# Then we use the bag-of-words representation on the 'about' column, saving only
# the words in the pre-defined list 'literature_genres'
bag_words_authors = pd.DataFrame(columns = literature_genres, index = df_authors_about.id)
# Set all entries to zero
bag_words_authors[bag_words_authors.isna()] = 0


def get_vector(description: str) -> Counter:
  """
  This function takes as input a string with the description of an author and
  counts how many time the literature_genres word appear in the description.
  The output is a Counter.
  """
  # Remove punctuation with a regex
  description = re.sub(r'[^\w\s]', '', description)
  description = description.lower()
  words = description.split()

  # Create the Counter
  word_counter = Counter(word for word in words if word in literature_genres)

  return word_counter

def fill_matrix(row) -> None:
  """
  This matrix fills the DataFrame bag_words_authors with the occurrence
  of the genre names
  """
  counter = get_vector(row['about'])

  if len(counter) != 0:
    tmp = pd.Series(0, index=literature_genres)
    tmp.update(counter)
    bag_words_authors.loc[row['id']] = tmp

# We use the apply function to improve the computation speed
df_authors_about.apply(fill_matrix, axis = 1)

In [ ]:
# Here we evaluate the percentage of descriptions that match at least one
# genre of the list 'literature_genres'
num_not_null = bag_words_authors[bag_words_authors.sum(axis = 1) != 0].shape[0]
percentage_not_null = num_not_null / bag_words_authors.shape[0] * 100
print(f"The percentage of authors that match at least one word: {round(percentage_not_null,2)}%")

# We drop all the records with no match
bag_words = bag_words_authors.drop(labels = bag_words_authors[bag_words_authors.sum(axis = 1) == 0].index, axis = 0)

The percentage of authors that match at least one word: 29.55%


In [ ]:
def get_genre(author_id: int) -> list:
  """
  This function takes as input the id of an author and returns a list
  with the genre that we associate to it.
  """
  try:
    row = bag_words.loc[author_id]
  except KeyError:
    return []

  maximum = max(row)
  indices_maximum = np.where(row == maximum)[0]
  return [x for x in bag_words.columns[indices_maximum]]

In [ ]:
# Here is an example where we ask for the literary genre of J.K. Rowling
# whose author id is 1077326

test_id = 1077326
genres = get_genre(test_id)
if len(genres) == 0:
  print(f"Unfortunately we have no idea for {df_authors[df_authors['id'] == test_id]['name'].tolist()[0]}")
else:
  print(f"Literary genres associated to {df_authors[df_authors['id'] == test_id]['name'].tolist()[0]}: {genres}")


Literary genres associated to J.K. Rowling: ['fantasy']


In [ ]:
# Add the literary genres to each author in a new column named 'literary_genre'

df_authors['literary_genre'] = df_authors.apply(lambda author: get_genre(author['id']), axis = 1)
df_authors.head(3)

,ratings_count,average_rating,text_reviews_count,work_ids,book_ids,works_count,id,name,gender,image_url,about,fans_count,literary_genre
0,2862064,4.19,62681,"[3078186, 135328, 1877624, 74123, 3078120, 104...","[386162, 13, 8695, 8694, 6091075, 365, 569429,...",106,4,Douglas Adams,male,https://images.gr-assets.com/authors/159137433...,"Douglas Noël Adams was an English author, comi...",19826,[novel]
1,1417316,4.02,84176,"[613469, 2305997, 940892, 2611786, 7800569, 31...","[9791, 21, 28, 24, 7507825, 27, 10538, 25, 26,...",75,7,Bill Bryson,male,https://images.gr-assets.com/authors/157859752...,"William McGuire ""Bill"" Bryson, OBE, FRS was bo...",16144,[]
2,56159,4.53,352,"[17150, 808427, 20487307, 90550, 25460625, 171...","[349254, 15222, 14833682, 15221, 18126815, 152...",14,10,Jude Fisher,female,https://images.gr-assets.com/authors/141145711...,"Jude Fisher is the pseudonym for <a href=""http...",60,"[fiction, fantasy, novel]"


---
# [AWSQ]

The answer to this research question requires to create an AWS EC2 instance. We decided to launch an instance with the following characteristics:

> * Amazon Machine Image:
`Ubuntu Server 22.04 LTS (HVM), SSD Volume Type, Architecture 64-bit (x86)`

> * Instance type: `t2.large (2 vCPU, 8 GiB Memory)`

> * Network settings: `Allow SSH traffic from anywhere (0.0.0.0/0)`

> * Storage: `20 GiB of gp2 (General Purpose SSD)`


In order to connect to the instance we enter the same directory where we downloaded the ssh key. Then then we open a terminal and run the following commands:

```
sudo chmod 400 "mykeypair.pem"
ssh -i "mykeypair.pem" ubuntu@ec2-52-0-31-203.compute-1.amazonaws.com
```

Now the SSH connection is open and we can work on the machine using the CLI.

First of all we have to update the software and to download the AWS CLI package:
```
sudo apt update && sudo apt upgrade
sudo apt install awscli
```
The EC2 instance still does not have pip and the python modules that we need to run the script, so we install them:
```
sudo apt install python3-pip
pip install pandas
```

Then we upload the 'list.json' file and the 'awsq.py' script (that provides the 5 most commonly used tags for book lists):
```
scp -i mykeypair.pem list.json ubuntu@ec2-52-0-31-203.compute-1.amazonaws.com:/home/ubuntu/
scp -i mykeypair.pem awsq.py ubuntu@ec2-52-0-31-203.compute-1.amazonaws.com:/home/ubuntu/
```

In the end we run the script with the following command:
```
python3 awsq.py
```

The outcome of the script is the following.
The top 5 most frequently used tags are:

| tag | # usage |
|:---:|:---:|
|	romance          |  6001 times|
|	fiction          |  5291 times|
| young-adult      |  5016 times|
|	fantasy          |  3666 times|
|	science-fiction  |  2779 times|

The script also outputs the execution time. The two execution times are:

| systems | time |
|:---:|:---:|
|	local ubuntu machine  |  69 seconds|
|	EC2 instance	      |  31 seconds|


We can see that the AWS EC2 instance is twice as fast as our local system.

Below is the python script 'awsq.py' used to compare the two systems:

In [ ]:
import pandas as pd
import time
import json
from collections import Counter

start_time = time.time()

chunks = pd.read_json("list.json", lines = True, chunksize = 100)

tag_counter = Counter()

for chk in chunks:
    tag_lists = chk['tags'].tolist()

    # tag_list is a list of lists, so we have to concatenate them.
    # We had to put an if statement in the list comprehension because
    # some items are not lists but float64 variables
    tag_list = [item.lower() for lst in tag_lists for item in (lst if hasattr(lst, '__iter__') else str(lst))]

    tag_counter.update(tag_list)

print("The top 5 most frequently used tags are:\n")
for elem in tag_counter.most_common(5):
    print(f"\t{elem[0]: <15}  :  {elem[1]} times")

execution_time = time.time() - start_time
print(f"\nExecution time: {round(execution_time)} seconds")